Để chạy được tutorial này, chúng ta cần phải cài đặt thư viện GenSim.
Để cài bằng Anaconda ta sử dụng lệnh sau:

$ conda install -c anaconda gensim

Sau đó import các thư viện cần thiết như sau:

In [18]:
# Test on 10 batches
iterations = 10
for i in range(iterations):
    next_batch, next_batch_labels = getTestBatch()
    # TODO 3.6: Tính độ chính xác 'accuracy' trên các test batch và gán vào 'test_acc'
    test_acc = sess.run(accuracy, {inputs: next_batch, labels: next_batch_labels}) * 100
    print("Accuracy for this batch:", test_acc)

Accuracy for this batch: 71.875
Accuracy for this batch: 71.875
Accuracy for this batch: 79.6875
Accuracy for this batch: 78.125
Accuracy for this batch: 81.25
Accuracy for this batch: 79.6875
Accuracy for this batch: 71.875
Accuracy for this batch: 73.4375
Accuracy for this batch: 84.375
Accuracy for this batch: 75.0


In [1]:
from gensim.models import KeyedVectors
import numpy as np

D:\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Load model
Trước hết, ta cần tải model về theo đường dẫn sau:https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki-news-300d-1M.vec.zip

Chi tiết có thể tham khảo trên trang FastText https://fasttext.cc/docs/en/english-vectors.html. Đối với các ngôn ngữ khác, trong đó có tiếng Việt ta có thể tải về từ đây:
https://fasttext.cc/docs/en/crawl-vectors.html

Thời gian load mô hình đã huấn luyện này có thể mất từ 5-10 phút )-:


In [2]:
word_vectors = KeyedVectors.load_word2vec_format('wiki-news-300d-1M.vec', binary=False)

FileNotFoundError: [Errno 2] No such file or directory: 'wiki-news-300d-1M.vec'

# Word embeding: biểu diễn một từ dưới dạng một vector

In [4]:
woman_vec = word_vectors['woman']
print('Feature representation of `woman` is:', woman_vec)
print('Size of feature representation: ', woman_vec.shape)

Feature representation of `woman` is: [  7.50999972e-02  -5.90999983e-02  -1.02000004e-02  -4.90000006e-03
  -2.22999994e-02   1.39699996e-01   1.97000001e-02   1.05800003e-01
  -6.74000010e-02   1.33300006e-01   8.54000002e-02   6.21999986e-02
   1.25000000e-01   8.06000009e-02   3.70000000e-03   2.58000009e-02
   7.55999982e-02   8.55000019e-02  -2.72000004e-02  -3.46999988e-02
  -1.16700001e-01  -9.99999975e-05  -1.72399998e-01  -1.09000001e-02
   1.63000003e-02  -1.14000002e-02  -3.55000012e-02   1.86900005e-01
   4.63000014e-02  -2.07000002e-02   6.83000013e-02   3.22999991e-02
   6.04000017e-02   1.78999994e-02  -5.79999983e-02  -3.86000015e-02
   1.89999994e-02   1.34000003e-01   1.24600001e-01   3.18999998e-02
   1.43000007e-01  -3.37000005e-02   6.50999993e-02  -1.36800006e-01
  -6.04999997e-02  -5.22999987e-02   3.33999991e-02   6.59999996e-02
  -8.39999970e-03  -7.98999965e-02   1.31799996e-01  -6.71999976e-02
  -6.14199996e-01   4.30000015e-03   1.47000002e-02  -6.62999973e

# Tính độ tương đồng giữa hai từ
Mức độ tương đồng ở đây là tổng thể từ loại, cách sử dụng từ và vai trò trong câu.
Do đó, đừng bất ngờ khi độ tương đồng của 'woman' và 'man' lại cao hơn 'woman' và 'girl'.
Vai trò, chức năng của 'woman' và 'man' trong câu thường có tính đối ngẫu gần giống nhau. Trong khi 'woman' và 'girl' thì không đối ngẫu, do 'girl' là một tập con của 'woman' nên trong cấu trúc câu từ 'girl' đôi khi không thể thay thế bằng 'woman' được.

In [10]:
print('Similarity of `woman` and `man`:', word_vectors.similarity('woman', 'man'))
print('Similarity of `woman` and `man`:', word_vectors.similarity('woman', 'girl'))
print('Similarity of `boy` and `man`:', word_vectors.similarity('boy', 'girl'))

Similarity of `woman` and `man`: 0.440141130997
Similarity of `woman` and `man`: 0.309353661372
Similarity of `boy` and `man`: 0.690931898928


# Mối quan hệ về không gian giữa các từ

Câu hỏi: Nếu như 'king' là 'man' thì 'queen' là gì?

Trong không gian vector để biểu diễn các từ, vị trí tương đối giữa các cặp từ có mối quan hệ tương đồng về ngữ nghĩa thì sẽ có khoảng cách tương đối giống nhau. Hình sau minh hoạ mối quan hệ về mặt không gian (vector) giữa các từ. Ta dự đoán rằng các vector biểu diễn này sẽ có sự tương đồng trong không gian, nghĩa là để trả lời câu hỏi trên, ta phải giải phương trình thể hiện mối quan hệ trong không gian như sau: 

vector('king') - vector('queen') = vector('man') - vector(X)

Suy ra: vector(X) = vector('man') - vector('king') + vector('queen').

Như vậy ta sẽ xác định xem những từ nào gần với phép biến đổi trên:
![title](king_and_queen.png)

In [6]:
sim_words = word_vectors.most_similar_cosmul(positive=['queen', 'man'], negative=['king'])
print('Queen is a: ', sim_words[0][0])

Queen is a:  woman


Như vậy, về mặt không gian vector, ta có thể biểu diễn được mối quan hệ ngữ nghĩa của các từ một cách đáng ngạc nhiên. Quan hệ trên là: Người đứng đầu - Giới tính.

Hoàn toàn tương tự, ta có thể khai thác được các mối quan hệ ngữ nghĩa khác như: Đất nước - Thủ Đô, Đất nước - món ăn, Công ty - Sản phẩm, Đất nước - Tổng thống. Ví dụ dưới đây minh hoạ cho mối quan hệ ngữ nghĩa: Đất nước - Thủ đô.

In [7]:
sim_words = word_vectors.most_similar_cosmul(positive=['Vietnam', 'Berlin'], negative=['Germany'])
print('If Berlin is the capital of Germany then', sim_words[0][0], 'is the capital of Vietnam')

If Berlin is the capital of Germany then Hanoi is the capital of Vietnam


In [11]:
sim_words

[('Hanoi', 0.9318509697914124),
 ('Saigon', 0.8905898332595825),
 ('Danang', 0.8561213612556458),
 ('Vietnamese', 0.854028046131134),
 ('VietNam', 0.8285140991210938),
 ('Hochiminh', 0.8211517930030823),
 ('Bangkok', 0.8155736327171326),
 ('Viet', 0.8127319812774658),
 ('Beijing', 0.8101763129234314),
 ('Hué', 0.80777907371521)]

# Thử nghiệm
Các bạn hãy thử trên các quan hệ khác như: Đất nước - món ăn, Công ty - Sản phẩm, Đất nước - Tổng thống, Bang - Thành phố.

In [2]:
word_vectors = KeyedVectors.load_word2vec_format('Vietnamese/cc.vi.300.vec', binary=False)

In [16]:
word_vectors.similarity('đất_nước', 'món_ăn')

0.34227028270042953

In [17]:
word_vectors.similarity('công_ty', 'sản_phẩm')

0.52512506211359444

In [18]:
word_vectors.similarity('đất_nước', 'tổng_thống')

0.41150791805130393

In [19]:
word_vectors.similarity('bang', 'thành_phố')

0.4258334697407441

In [39]:
sim_words = word_vectors.most_similar_cosmul(positive=['nhà_vua', 'phụ_nữ'], negative=['đàn_ông'])
sim_words

[('Nhà_vua', 0.9613238573074341),
 ('Nhà_Vua', 0.8581167459487915),
 ('nhà_Vua', 0.8480479121208191),
 ('vua', 0.8382401466369629),
 ('Hoàng_hậu', 0.8351410031318665),
 ('hoàng_hậu', 0.832502543926239),
 ('triều_đình', 0.8315482139587402),
 ('triều_thần', 0.8289474844932556),
 ('Quốc_vương', 0.8155739307403564),
 ('Vua_Chúa', 0.8079200387001038)]

In [29]:
word_vectors['sài_gòn']

array([ 0.045 , -0.0254, -0.0449, -0.0241,  0.0116, -0.0127,  0.0003,
       -0.0381, -0.0559,  0.0013, -0.0056, -0.1131, -0.0473, -0.0283,
       -0.0533, -0.0038, -0.021 , -0.0076,  0.038 ,  0.0371, -0.0358,
        0.0629, -0.0421,  0.0289,  0.0072,  0.011 ,  0.0215, -0.029 ,
        0.0548, -0.0301, -0.0339, -0.0356, -0.0408, -0.0506, -0.0383,
       -0.0126, -0.0334, -0.0349, -0.0205, -0.002 ,  0.0428,  0.0067,
        0.0127, -0.0271,  0.028 ,  0.0308,  0.0108, -0.0532, -0.0161,
        0.0468,  0.1274, -0.1041, -0.0495,  0.0645, -0.0603,  0.0144,
       -0.0518, -0.0479, -0.0186, -0.0095,  0.0781,  0.0245, -0.0336,
       -0.0518, -0.0281, -0.0226, -0.0502, -0.0426,  0.0221,  0.0028,
        0.0309, -0.0222,  0.    ,  0.0494,  0.0503,  0.0294, -0.0467,
        0.0357, -0.0098,  0.036 , -0.0461, -0.051 , -0.0017, -0.0654,
        0.0113, -0.0269, -0.0273,  0.035 ,  0.0852,  0.0487,  0.0249,
        0.012 ,  0.0252,  0.0105,  0.0349,  0.0358, -0.0102,  0.0663,
        0.0084,  0.0